In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_rel
import matplotlib.ticker as mtick
import utils
import gc
import importlib
import numba
from statsmodels.api import add_constant
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
numba.set_num_threads(8)
importlib.reload(utils)
gc.collect()

In [ ]:
firm_data_daily = pd.read_csv('data/firm_daily_returns_cleaned.csv', low_memory=False)
gc.collect()

In [ ]:
df = pd.read_csv('data_merged_pctfluidshiftpct.csv')
ff_data = pd.read_csv('data/ff5_data_daily.csv')

In [ ]:
df.columns

In [ ]:
def plot_amalgam_vs_index(amalgam_returns, index_returns):
    amalgam_returns.index = pd.to_datetime(amalgam_returns.index)
    index_returns.index = pd.to_datetime(index_returns.index)

    shared_index = amalgam_returns.index.intersection(index_returns.index)
    tmp = pd.DataFrame({
        'Amalgam': amalgam_returns.loc[shared_index],
        'Index': index_returns.loc[shared_index]
    })

    tmp = tmp + 1
    tmp = tmp.cumprod()

    max_val = tmp.max().max()
    yticks_vals = [1]
    base = 2
    while base <= max_val:
        yticks_vals.append(base)
        base *= 2

    plt.figure(figsize=(10, 6))  
    plt.plot(tmp.index, tmp['Amalgam'], label='Amalgam Model', linewidth=0.9)
    plt.plot(tmp.index, tmp['Index'], label='Index', linewidth=0.9)
    plt.legend()

    plt.yscale('log')
    plt.yticks(yticks_vals, [f"{int(y*100)}%" for y in yticks_vals])
    plt.title("Cumulative Performance")
    plt.tight_layout()
    plt.show()

In [ ]:
ff_data['date'] = pd.to_datetime(ff_data['date'].astype(str), format='%Y%m%d')
ff_data.set_index('date', inplace=True)

In [ ]:
index_data_daily = pd.read_csv('data/index_daily_returns.csv')
index_data_daily['date'] = pd.to_datetime(index_data_daily['date'], dayfirst=True)
index_data_daily = index_data_daily[index_data_daily['date'].dt.year >= 1980]
index_data_daily = index_data_daily[index_data_daily['date'].dt.year <= 2025]
index_data_daily.set_index('date', inplace=True)
returns = pd.DataFrame()
returns['VWTD_Index'] = index_data_daily['vwretd']

### Number of Observations, companies with MC =/> 50 million

In [ ]:
years = df.query('y >= 1961')['y']
sns.histplot(years, bins=years.nunique())

plt.xlabel('Year')
plt.ylabel('Firm-Quarter Observations')
plt.gca().yaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{int(x):,}'))
plt.tight_layout()
plt.show()
gc.collect()

#### Modified Label

In [ ]:
df['GIND'] = df['GIND'].astype("category")  
features = [
    'cop_at_pct', 'noa_gr1a_pct', 'saleq_gr1_pct', 'resff3_12_1_pct',
    'seas_6_10an_pct', 'debt_me_pct', 'seas_6_10na_pct',
    'zero_trades_252_pct', 'zero_trades_21_pct', 'cowc_gr1a_pct',
    'nncoa_gr1a_pct', 'ocf_me_pct', 'turnover_126d_pct',
    'rmax5_rvol_21d_pct', 'seas_11_15na_pct', 'o_score_pct', 'niq_at_pct',
    'seas_20_16an_pct', 'ni_arl_pct', 'ivol_ff3_21d_pct', 'ni_me_pct',
    'dsale_dinv_pct', 'ni_be_pct', 'noa_at_pct', 'firm_age_pct',
    'mom_12_1_pct', 'nfna_grla_pct', 'at_me_pct', 'GIND'
]

In [ ]:
# Drop missing RET or MC
df = df.dropna(subset=['RET', 'MC']).copy()

# RET percentile within GIND per quarter
df['ret_pct_gind'] = (
    df.groupby(['GIND', 'y', 'qtr'])['RET']
    .transform(lambda x: x.rank(pct=True))
)

# Value-weighted RET percentile across all firms per quarter
# First, compute RET percentiles across all firms (not GIND-specific)
df['ret_pct_all'] = (
    df.groupby(['y', 'qtr'])['RET']
    .transform(lambda x: x.rank(pct=True))
)

# VWRETD_PCT per quarter
vwretd_pct = (
    df.groupby(['y', 'qtr'])
    .apply(lambda g: (g['ret_pct_all'] * g['MC']).sum() / g['MC'].sum())
    .rename('vwretd_pct')
    .reset_index()
)

# Merge back to df
df = df.merge(vwretd_pct, on=['y', 'qtr'], how='left')

#Step 3: Label assignment
df['label'] = 0
df.loc[(df['ret_pct_gind'] > 0.5) & (df['ret_pct_all'] > df['vwretd_pct']), 'label'] = 1

In [ ]:
from utils import XgboostRolling  

df = df.replace([np.inf, -np.inf], np.nan) 

xgb_model = XgboostRolling(
    data=df,
    features=features,          
    label='label',               
    test_y_qtr_cut=(1981, 1),    
    rolling_interval=1,           
    last_y_qtr=(2024,4)
)

xgb_model.fit(tuning_method='sequential')  

### Save Model to directory

In [ ]:
import os
os.makedirs("models/amalgam_xgb_roll", exist_ok=True)
for i, (y, q) in enumerate(xgb_model.lst_rolling_cut):
    try:
        model = xgb_model.models[i]
        model.save_model(f"models/amalgam_xgb_roll/xgb_{y}_{q}.json")
    except IndexError:
        print(f"No model trained for cut: {(y, q)} — skipping.")

gc.collect()

#### Predictions + Top Decile Returns

In [ ]:
# Get predictions
testset = xgb_model.get_testset_with_prob()

# Assign top decile label
testset['ML_is_top_decile'] = testset.groupby(['y', 'qtr'])['prob'].transform(
    lambda x: x >= x.quantile(0.9)
)

# Drop old Amalgam column if it exists
if 'Amalgam' in firm_data_daily.columns:
    firm_data_daily = firm_data_daily.drop(columns=['Amalgam'])

# Merge testset into firm_data_daily
merged_parts = []
unique_quarters = firm_data_daily[['y', 'qtr']].drop_duplicates().sort_values(['y', 'qtr']).values.tolist()

for y, q in unique_quarters:
    firm_chunk = firm_data_daily[(firm_data_daily['y'] == y) & (firm_data_daily['qtr'] == q)]
    test_chunk = testset[testset['y'].eq(y) & testset['qtr'].eq(q)][['y', 'qtr', 'PERMNO', 'ML_is_top_decile']]

    merged = pd.merge(
        firm_chunk,
        test_chunk.rename(columns={'ML_is_top_decile': 'Amalgam'}),
        on=['y', 'qtr', 'PERMNO'],
        how='left'
    )
    merged_parts.append(merged)

firm_data_daily = pd.concat(merged_parts, ignore_index=True)
gc.collect()

In [ ]:
testset.to_csv('data/Amalgam_Top_Predictions.csv', index=False)
firm_data_daily['Amalgam'] = firm_data_daily['Amalgam'].fillna(False)
ML_returns = firm_data_daily[firm_data_daily['Amalgam']].groupby(['date'])['RET'].mean()
plot_amalgam_vs_index(ML_returns, returns['VWTD_Index'])
ML_returns.to_csv("data/Amalgam_Daily_Returns.csv")
gc.collect()

#### Bottom Decile + Top minus Bottom Decile Returns

In [ ]:
if 'testset' not in locals():
    testset = xgb_model.get_testset_with_prob()

# Identify bottom decile
testset['ML_is_bottom_decile'] = testset.groupby(['y', 'qtr'])['prob'].transform(
    lambda x: x <= x.quantile(0.1)
)

if 'Amalgam_Bottom' in firm_data_daily.columns:
    firm_data_daily = firm_data_daily.drop(columns=['Amalgam_Bottom'])

# Merge bottom decile into firm_data_daily
merged_parts = []
for y, q in firm_data_daily[['y', 'qtr']].drop_duplicates().values.tolist():
    firm_chunk = firm_data_daily[(firm_data_daily['y'] == y) & (firm_data_daily['qtr'] == q)]
    bottom_chunk = testset[testset['y'].eq(y) & testset['qtr'].eq(q)][['y', 'qtr', 'PERMNO', 'ML_is_bottom_decile']]

    merged = pd.merge(
        firm_chunk,
        bottom_chunk.rename(columns={'ML_is_bottom_decile': 'Amalgam_Bottom'}),
        on=['y', 'qtr', 'PERMNO'],
        how='left'
    )
    merged_parts.append(merged)

firm_data_daily = pd.concat(merged_parts, ignore_index=True)
firm_data_daily['Amalgam_Bottom'] = firm_data_daily['Amalgam_Bottom'].fillna(False)

# Calculate returns
ML_top_returns = firm_data_daily[firm_data_daily['Amalgam']].groupby('date')['RET'].mean()
ML_bottom_returns = firm_data_daily[firm_data_daily['Amalgam_Bottom']].groupby('date')['RET'].mean()
ML_TMB_returns = ML_top_returns - ML_bottom_returns

# Save
ML_bottom_returns.to_csv("data/Amalgam_Bottom_Daily_Returns.csv")
ML_TMB_returns.to_csv("data/Amalgam_TopMinusBottom_Daily_Returns.csv")
gc.collect()